# Calculation of M/H2/n Queueing System with H2-Warming

### On the Warming Mode
The "warming" mode is activated if the system was empty (no requests were being served and no requests were in queue) and a new request arrived. In this case, the service for the arriving request will start after a random period of "warming" for the system.
Additionally, all incoming requests during the "warming" period of the system are placed in the queue and are not serviced until the end of this period.

### About the Algorithm
In this implementation, it is assumed that the service time and warming time follow *H2*-distributions, each with its own parameters. The parameters of the *H2*-distributions are selected based on the specified average values and coefficients of variation.

The algorithm allows calculating the system for any coefficient of variation in service time. Even when the coefficient of variation in service time is less than 1, the parameters of the approximating *N2*-distribution can be complex, which does not prevent obtaining meaningful results.

For verification, simulation modeling is used.

### Set paramrameters

In [5]:
from most_queue.theory.vacations.m_h2_h2warm import MH2nH2Warm
from most_queue.sim.vacations import VacationQueueingSystemSimulator
from most_queue.rand_distribution import GammaDistribution
from most_queue.general.tables import probs_print, times_print

import math
import time

n = 5  # number of channels # Number of channels
l = 1.0  # Intensity of the input stream # Intensity of the input stream
ro = 0.7  # Traffic intensity # Traffic intensity
b1 = n * ro  # Average service time # Average service time
b1_warm = n * 0.2  # Average warming time # Average warming time
num_of_jobs = 1000000  # Number of jobs to be served by IM # Number of jobs to be served by IM
b_coev = 1.1  # Coefficient of variation of service time # Coefficient of variation of service time
b_coev_warm = 1.3  # Coefficient of variation of warming time # Coefficient of variation of warming time
verbose = False  # Do not print explanations during calculations # Do not print explanations during calculations


### Set initial moments of service time

In [6]:
b = [0.0] * 3
alpha = 1 / (b_coev ** 2)
b[0] = b1
b[1] = math.pow(b[0], 2) * (math.pow(b_coev, 2) + 1)
b[2] = b[1] * b[0] * (1.0 + 2 / alpha)

### Set warm-up moments of service time

In [7]:
b_w = [0.0] * 3
b_w[0] = b1_warm
alpha = 1 / (b_coev_warm ** 2)
b_w[1] = math.pow(b_w[0], 2) * (math.pow(b_coev_warm, 2) + 1)
b_w[2] = b_w[1] * b_w[0] * (1.0 + 2 / alpha)

### Run simulation

In [8]:
sim_start = time.process_time()

qs = VacationQueueingSystemSimulator(n)  # queueing system simulator object
qs.set_sources(l, 'M')  # set sources with exponential distribution

gamma_params = GammaDistribution.get_params(b)
qs.set_servers(gamma_params, 'Gamma')  # set servers with Gamma distribution

gamma_params_warm = GammaDistribution.get_params(b_w)
qs.set_warm(gamma_params_warm, 'Gamma')  # set warm-up servers with Gamma distribution

qs.run(num_of_jobs)  # run simulation

# After simulation is complete, save the probabilities of states and start times of service.
p = qs.get_p()
v_sim = qs.v

sim_time = time.process_time() - sim_start

Start simulation


Job served:    | 0/100 [00:00<?, ?it/s]10000/1000000:   1%|          | 1/100 [00:00<00:08, 11.20it/s]10000/1000000:   2%|▏         | 2/100 [00:00<00:08, 11.69it/s]20000/1000000:   2%|▏         | 2/100 [00:00<00:08, 11.69it/s]30000/1000000:   3%|▎         | 3/100 [00:00<00:08, 11.69it/s]30000/1000000:   4%|▍         | 4/100 [00:00<00:08, 11.40it/s]40000/1000000:   4%|▍         | 4/100 [00:00<00:08, 11.40it/s]50000/1000000:   5%|▌         | 5/100 [00:00<00:08, 11.40it/s]50000/1000000:   6%|▌         | 6/100 [00:00<00:08, 11.52it/s]60000/1000000:   6%|▌         | 6/100 [00:00<00:08, 11.52it/s]70000/1000000:   7%|▋         | 7/100 [00:00<00:08, 11.52it/s]70000/1000000:   8%|▊         | 8/100 [00:00<00:08, 11.39it/s]80000/1000000:   8%|▊         | 8/100 [00:00<00:08, 11.39it/s]90000/1000000:   9%|▉         | 9/100 [00:00<00:07, 11.39it/s]90000/1000000:  10%|█         | 10/100 [00:00<00:07, 11.44it/s]100000/1000000:  10%|█         | 10/100 [00:00<00:07, 11.44it/s]110000/1000000:  11%|█      

Simulation is finished



### Run numerical method

In [9]:
tt_start = time.process_time()
tt = MH2nH2Warm(l, b, b_w, n, verbose=verbose)  # create object of the numerical method

tt.run()  # run numerical method

# After the calculation is complete, save the probabilities of states and start times of service.
p_tt = tt.get_p()
v_tt = tt.get_v()

tt_time = time.process_time() - tt_start

num_of_iter = tt.num_of_iter_  # number of iterations

/home/roman/py_proj/most_queue_git/most-queue/most_queue/theory/vacations/m_h2_h2warm.py:286: ComplexWarning: Casting complex values to real discards the imaginary part
  while math.fabs(1.0 - p_sum) > 1e-6:


### Print results

In [10]:
print("\nComparison of results calculated by the Takacs-Takaichi method and simulation.\n"
      f"Sim - M/Gamma/{n:^2d} with Gamma warming\n"
      f"Takacs-Takaichi - M/H2/{n:^2d} with H2 warming"
      f"Load coefficient: {ro:^1.2f}")
print(f'Variation coefficient of service time {b_coev:0.3f}')
print(f'Variation coefficient of warming time {b_coev_warm:0.3f}')
print(f"Number of iterations of the Takacs-Takaichi algorithm: {num_of_iter:^4d}")
print(f"Takacs-Takaichi algorithm runtime: {tt_time:^5.3f} s")
print(f"Sim runtime: {sim_time:^5.3f} s")

probs_print(p, p_tt, 10)

times_print(v_sim, v_tt, is_w=False)


Comparison of results calculated by the Takacs-Takaichi method and simulation.
Sim - M/Gamma/5  with Gamma warming
Takacs-Takaichi - M/H2/5  with H2 warmingLoad coefficient: 0.70
Variation coefficient of service time 1.100
Variation coefficient of warming time 1.300
Number of iterations of the Takacs-Takaichi algorithm:  26 
Takacs-Takaichi algorithm runtime: 0.381 s
Sim runtime: 9.804 s
------------------------------------
      Probabilities of states       
------------------------------------
 #  |      Num      |      Sim      
------------------------------------
 0  |   0.021523    |   0.021311    
 1  |   0.086511    |   0.086953    
 2  |    0.15644    |    0.15704    
 3  |    0.18445    |    0.18413    
 4  |    0.16128    |    0.16099    
 5  |    0.11115    |    0.11002    
 6  |   0.078321    |   0.077805    
 7  |   0.055874    |   0.055841    
 8  |   0.040129    |   0.040328    
 9  |   0.028924    |    0.02929    
------------------------------------


Initial moment